In [1]:
from google.colab import files

upl = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
!ls

kaggle.json  sample_data


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
upl = files.upload()

In [7]:
!unzip facial-expression-recognition-challenge.zip -d  facial-expression-recognition-challenge

Archive:  facial-expression-recognition-challenge.zip
  inflating: facial-expression-recognition-challenge/example_submission.csv  
  inflating: facial-expression-recognition-challenge/face_embs.npy  
  inflating: facial-expression-recognition-challenge/fer2013.tar  
  inflating: facial-expression-recognition-challenge/icml_face_data.csv/icml_face_data.csv  
  inflating: facial-expression-recognition-challenge/metadata_processed.csv  
  inflating: facial-expression-recognition-challenge/test.csv/test.csv  


In [8]:
import pandas as pd

df = pd.read_csv('facial-expression-recognition-challenge/metadata_processed.csv')
emoticon = {'Angry': 0, 'Disgust' : 1, 'Fear' : 2, 'Happy' :3, 'Sad': 4, 'Surprise' : 5, 'Neutral': 6}
df['emotion'] = df['emotion'].map(emoticon)
df.head(5)

,label,image,emotion
0,4,59 71 89 118 139 160 183 222 212 148 77 16 16 ...,4
1,0,144 149 129 114 119 124 125 135 145 141 147 15...,0
2,4,90 97 105 42 41 41 37 40 36 55 104 128 145 162...,4
3,1,69 72 80 87 95 102 112 123 131 137 143 151 159...,1
4,0,134 135 136 137 142 122 46 47 36 25 21 18 13 1...,0


In [9]:

import torch
import numpy as np

def process_image(image_string):
    # Split the string into a list of integers
    pixel_values = np.fromstring(image_string, dtype=int, sep=' ')

    # Reshape into a 48x48 array (assuming the images are 48x48)
    image_array = pixel_values.reshape(48, 48)

    # Convert to a PyTorch tensor
    tensor_image = torch.tensor(image_array, dtype=torch.float32)

    # Normalize the pixel values to the range [0, 1] if needed
    tensor_image /= 255.0

    return tensor_image

df['image'] = df['image'].apply(process_image)

In [10]:
df.head()
df['image'].shape

(35882,)

In [11]:
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

# transform = transforms.Compose([
#     transforms.Resize((28, 28)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5], [0.5])
# ])

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.Normalize([0.5], [0.5])
])

image_tensors = torch.stack(df['image'].tolist())
image_tensors = image_tensors.view(-1, 1, 48, 48)
label_tensors = torch.tensor(df['emotion'].tolist(), dtype=torch.long)

dataset = TensorDataset(image_tensors, label_tensors)


In [12]:
class TransformDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.dataset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.dataset)

from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_dataset = TransformDataset(train_dataset, transform)
valid_dataset = TransformDataset(test_dataset, transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")


Number of training samples: 28705
Number of testing samples: 7177


In [13]:
!ls

facial-expression-recognition-challenge      kaggle.json
facial-expression-recognition-challenge.zip  sample_data


In [20]:
from torch import nn

lr = 0.001
n_ch =1

model = nn.Sequential()

#featurizer
model.add_module('c1', nn.Conv2d(
    in_channels=1, out_channels=n_ch, kernel_size=3
))
model.add_module('relu1', nn.ReLU())
model.add_module('maxp1', nn.MaxPool2d(kernel_size=2))
model.add_module('flatten', nn.Flatten())

#classfication
model.add_module('fc1', nn.Linear(n_ch*13*13, 70))
model.add_module('relu2', nn.ReLU())
model.add_module('fc3', nn.Linear(70, 35))
model.add_module('relu3', nn.ReLU())
model.add_module('fc4', nn.Linear(35, 12))
model.add_module('relu4', nn.ReLU())
model.add_module('fc2', nn.Linear(12, 7))

model

Sequential(
  (c1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (maxp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=169, out_features=70, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=70, out_features=35, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=35, out_features=12, bias=True)
  (relu4): ReLU()
  (fc2): Linear(in_features=12, out_features=7, bias=True)
)

In [21]:
import torch

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
torch.manual_seed(1)
num_epochs = 50
log_epoch = 5

for epoch in range(num_epochs):
  accuracy_hist_train = 0
  for images, labels in train_loader:
    pred = model(images)
    loss = loss_fn(pred, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    is_correct = (
        torch.argmax(pred, dim=1) == labels
        ).float()
    accuracy_hist_train += is_correct.sum()
  accuracy_hist_train /= len(train_loader.dataset)
  if (epoch % log_epoch == 0):
    print(f'Epoch {epoch + 1} Accuracy '
    f'{accuracy_hist_train:.4f}')

Epoch 1 Accuracy 0.2477
Epoch 6 Accuracy 0.4270
Epoch 11 Accuracy 0.4870
Epoch 16 Accuracy 0.5189
Epoch 21 Accuracy 0.5422
Epoch 26 Accuracy 0.5635
Epoch 31 Accuracy 0.5837
Epoch 36 Accuracy 0.5987
Epoch 41 Accuracy 0.6098
Epoch 46 Accuracy 0.6277


In [22]:
from google.colab import files

ans = files.upload()

Saving sad.jpeg to sad.jpeg


In [25]:
from PIL import  Image

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

image = Image.open('sad.jpeg')
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

output = model(image_tensor)
output

tensor([[  2.1164, -25.9275,  -0.7778,   2.3993,   2.6792,  -4.5614,   2.4400]],
       grad_fn=<AddmmBackward0>)

In [26]:
torch.save(model.state_dict(), 'facial_emotion.ph')

In [28]:
model.state_dict()

OrderedDict([('c1.weight',
              tensor([[[[ 0.2856, -1.6876,  0.5857],
                        [-0.8934, -0.7841, -0.0956],
                        [ 0.9468,  0.9653, -0.4577]]]])),
             ('c1.bias', tensor([0.2809])),
             ('fc1.weight',
              tensor([[-0.1515,  0.1750,  0.0928,  ..., -0.0988,  0.2175,  0.2046],
                      [ 0.2790,  0.0098,  0.0942,  ...,  0.2133,  0.0073, -0.0461],
                      [ 0.3790,  0.0252, -0.1205,  ..., -0.1836,  0.2561, -0.1155],
                      ...,
                      [ 0.2517,  0.2924,  0.2205,  ..., -0.0788, -0.2180, -0.0305],
                      [-0.2272, -0.2113, -0.0793,  ..., -0.2275,  0.0845, -0.5135],
                      [-0.0169, -0.0587,  0.0427,  ..., -0.0930, -0.0861, -0.3494]])),
             ('fc1.bias',
              tensor([ 0.2998, -0.0417, -0.2051, -0.5162, -0.2020, -0.2206, -0.0862,  0.0325,
                       0.1138,  0.0866, -0.1578, -0.4352,  0.0909, -0.0329,  0.2853

In [29]:
files.download('facial_emotion.ph')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>